# Definition

Grey Wolf Optimization (GWO) is a metaheuristic optimization algorithm that is inspired by the hunting behavior of grey wolves in the wild. It was introduced by Seyedali Mirjalili, Seyed Mohammad Mirjalili, and Andrew Lewis in 2014.

The algorithm is based on a hierarchy of four types of wolves: alpha, beta, delta, and omega. Alpha wolves are the most dominant, followed by beta, delta, and omega in decreasing order of dominance. In GWO, each wolf represents a candidate solution to the optimization problem.

The GWO algorithm works as follows:

Initialize the population of wolves with random solutions.
Evaluate the fitness of each wolf.
Determine the hierarchy of wolves (i.e., alpha, beta, delta, and omega).
Update the position of each wolf based on its hierarchy and the positions of the other wolves.
Apply bounds to the position of each wolf to ensure that it remains within the search space.
Repeat steps 2-5 until a stopping criterion is met.
The position of each wolf is updated using the following equations:

For the alpha wolf:
D_alpha = abs(C1 * X_alpha - positions), where C1 is a constant and X_alpha is the position of the alpha wolf.
A1 = 2 * a * r1 - a, where a is a coefficient, r1 is a random number in [0,1], and * denotes element-wise multiplication.
D_alpha_star = A1 * D_alpha
X1 = X_alpha - D_alpha_star
For the beta and delta wolves:
D_beta = abs(C1 * X_beta - positions), where X_beta is the position of the beta or delta wolf.
A2 = 2 * a * r1 - a
D_beta_star = A2 * D_beta
X2 = X_beta - D_beta_star
For the omega wolf:
D_omega = abs(C1 * X_omega - positions), where X_omega is the position of the omega wolf.
A3 = 2 * a * r1 - a
D_omega_star = A3 * D_omega
X3 = X_omega - D_omega_star
where positions are the positions of all the wolves, and C1 and a are constants.

After updating the positions of the wolves, the fitness of each wolf is evaluated, and the hierarchy is determined based on the fitness values. The alpha, beta, and delta wolves become the new reference positions for the next iteration.

The GWO algorithm is a population-based optimization algorithm that can be applied to a wide range of optimization problems. It has been shown to be effective in solving both unconstrained and constrained optimization problems, as well as multi-objective optimization problems.

# Codes

In [1]:
import numpy as np

# Define the benchmark functions
def sphere(x):
    return np.sum(x**2)

def rastrigin(x):
    d = len(x)
    return 10*d + np.sum(x**2 - 10*np.cos(2*np.pi*x))

def griewank(x):
    d = len(x)
    s = np.sum(x**2)
    p = np.prod(np.cos(x / np.sqrt(np.arange(1, d+1))))
    return 1 + s/4000 - p

# Define the GWO function
def gwo(f, lb, ub, dim, SearchAgents_no, Max_iter):
    # Initialize the positions of the wolves
    Positions = np.random.uniform(lb, ub, size=(SearchAgents_no, dim))
    # Initialize the best solution found so far
    Best_wolf = np.zeros(dim)
    Best_score = np.inf
    # Main loop
    for i in range(Max_iter):
        # Update the positions of the wolves
        a = 2 - 2 * i / Max_iter
        for j in range(SearchAgents_no):
            # Calculate the fitness of the current wolf
            fitness = f(Positions[j])
            # Update the best solution found so far
            if fitness < Best_score:
                Best_score = fitness
                Best_wolf = Positions[j]
            # Update the positions of the wolves
            A1 = 2 * a * np.random.rand(dim) - a
            C1 = 2 * np.random.rand(dim)
            D_alpha = np.abs(C1 * Best_wolf - Positions[j])
            X1 = Best_wolf - A1 * D_alpha
            
            A2 = 2 * a * np.random.rand(dim) - a
            C2 = 2 * np.random.rand(dim)
            D_beta = np.abs(C2 * Positions[np.random.randint(SearchAgents_no)] - Positions[j])
            X2 = Positions[np.random.randint(SearchAgents_no)] - A2 * D_beta
            
            A3 = 2 * a * np.random.rand(dim) - a
            C3 = 2 * np.random.rand(dim)
            D_delta = np.abs(C3 * Positions[np.random.randint(SearchAgents_no)] - Positions[j])
            X3 = Positions[np.random.randint(SearchAgents_no)] - A3 * D_delta
            
            Positions[j] = (X1 + X2 + X3) / 3
            # Apply bounds to the position of the current wolf
            Positions[j] = np.clip(Positions[j], lb, ub)
            
    # Return the best solution found
    return Best_wolf, Best_score

In [2]:
# Test the GWO function on the benchmark functions
dim = 4
SearchAgents_no = 50
Max_iter = 150
lb = -100
ub = 100

print("Solving the sphere function...")
Best_wolf, Best_score = gwo(sphere, lb, ub, dim, SearchAgents_no, Max_iter)
print("Best solution found:", Best_wolf)
print("Best score:", Best_score)

print("Solving the rastrigin function...")
Best_wolf, Best_score = gwo(rastrigin, lb, ub, dim, SearchAgents_no, Max_iter)
print("Best solution found:", Best_wolf)
print("Best score:", Best_score)

print("Solving the griewank function...")
Best_wolf, Best_score = gwo(griewank, lb, ub, dim, SearchAgents_no, Max_iter)
print("Best solution found:", Best_wolf)
print("Best score found:", Best_score)

Solving the sphere function...
Best solution found: [-45.147439   -40.13122015  -0.6658136   -3.32229189]
Best score: 192.3495137990838
Solving the rastrigin function...
Best solution found: [-20.16540685  13.52455706   4.89342095   0.69204125]
Best score: 63.29228801473302
Solving the griewank function...
Best solution found: [-1.23721358  2.58157477 18.34240315 -0.20795936]
Best score found: 0.1820040214967148


## Functional Programming Style

In [3]:
import numpy as np

def sphere(x):
    return sum(x**2)

def rastrigin(x):
    A = 10
    return A*len(x) + sum(x**2 - A*np.cos(2*np.pi*x))

def rosenbrock(x):
    return sum(100*(x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

def initialize_search_agents(search_agent_no, lb, ub, dim):
    return np.random.uniform(lb, ub, (search_agent_no, dim))

def update_alpha_beta_delta(search_agents, function):
    sorted_idx = np.argsort([function(agent) for agent in search_agents])
    alpha, beta, delta = search_agents[sorted_idx[:3]]
    return alpha, beta, delta

def update_positions(search_agents, alpha, beta, delta, a, lb, ub):
    r1 = np.random.random(search_agents.shape)  # random vector
    r2 = np.random.random(search_agents.shape)  # random vector
    A1 = 2 * a * r1 - a  # coefficient vector
    C1 = 2 * r2  # coefficient vector
    D_alpha = abs(C1 * alpha - search_agents)
    X1 = alpha - A1 * D_alpha

    r1 = np.random.random(search_agents.shape)  # random vector
    r2 = np.random.random(search_agents.shape)  # random vector
    A2 = 2 * a * r1 - a  # coefficient vector
    C2 = 2 * r2  # coefficient vector
    D_beta = abs(C2 * beta - search_agents)
    X2 = beta - A2 * D_beta

    r1 = np.random.random(search_agents.shape)  # random vector
    r2 = np.random.random(search_agents.shape)  # random vector
    A3 = 2 * a * r1 - a  # coefficient vector
    C3 = 2 * r2  # coefficient vector
    D_delta = abs(C3 * delta - search_agents)
    X3 = delta - A3 * D_delta

    # update the positions of the search agents
    new_search_agents = X1 + X2 + X3
    new_search_agents = np.clip(new_search_agents, lb, ub)
    return new_search_agents

def GWO(function, lb, ub, dim, search_agent_no, max_iter):
    # initialize the search agents
    search_agents = initialize_search_agents(search_agent_no, lb, ub, dim)
    alpha, beta, delta = update_alpha_beta_delta(search_agents, function)

    convergence_curve = np.zeros(max_iter)

    # main loop
    for iter_no in range(max_iter):
        a = 2 - 2 * iter_no / max_iter  # linearly decreased from 2 to 0

        # update the positions of the search agents
        search_agents = update_positions(search_agents, alpha, beta, delta, a, lb, ub)

        # update alpha, beta, and delta positions
        alpha, beta, delta = update_alpha_beta_delta(search_agents, function)

        # update convergence curve
        convergence_curve[iter_no] = function(alpha)

    return alpha, function(alpha), convergence_curve
